In [ ]:
pip install yfinance

In [2]:
import pandas as pd
import requests
from pycoingecko import CoinGeckoAPI
from pprint import pprint
from binance import Client
from datetime import datetime
import yfinance as yf

In [3]:
cg = CoinGeckoAPI()
client = Client()

Se decide hacer un análisis sobre la categoría Capa 1 (L1) de criptomonedas por su capitalización de mercado.

Categorías de Interes

In [4]:
Categorias=['Layer 1 (L1)']

In [5]:
#Lista de todas las categorías
categoria=pd.DataFrame(cg.get_coins_categories_list())
categoria[categoria['name'].isin(Categorias)]

,category_id,name
105,layer-1,Layer 1 (L1)


In [6]:
#Lista de todas las categorías con datos de mercado
datos_merc_ct=pd.DataFrame(cg.get_coins_categories(order='market_cap_asc'))
datos_merc_ct=datos_merc_ct.sort_values(by='market_cap',ascending=False).dropna(subset='market_cap')
datos_merc_ct.head()

,id,name,market_cap,market_cap_change_24h,content,top_3_coins,volume_24h,updated_at
114,layer-1,Layer 1 (L1),7.903085e+11,-1.230963,,[https://assets.coingecko.com/coins/images/1/s...,2.097109e+10,2023-08-25T01:40:19.396Z
113,smart-contract-platform,Smart Contract Platform,3.053585e+11,-1.332824,Smart contract platforms are usually blockchai...,[https://assets.coingecko.com/coins/images/279...,1.031964e+10,2023-08-25T01:40:05.125Z
112,stablecoins,Stablecoins,1.240874e+11,0.071245,,[https://assets.coingecko.com/coins/images/325...,2.403005e+10,2023-08-25T01:40:04.850Z
111,alleged-sec-securities,Alleged SEC Securities,8.271318e+10,-1.004503,Tokens and cryptocurrencies alleged to be secu...,[https://assets.coingecko.com/coins/images/825...,3.486695e+09,2023-08-25T01:40:03.116Z
110,exchange-based-tokens,Exchange-based Tokens,5.298405e+10,-0.405979,These tokens are normally associated with both...,[https://assets.coingecko.com/coins/images/825...,9.326706e+08,2023-08-25T01:40:06.809Z


In [7]:
def get_coin_list():
    return cg.get_coins_list()

In [8]:
df1=get_coin_list()
df1=pd.DataFrame(df1)

In [9]:
#crypto_symbol=['btc','eth','bnb','ada','sol','usdt','usdc','dai','busd','tusd']
df1[df1['name'].str.contains('TrueUS', case=False)]


,id,symbol,name
1474,bridged-trueusd,tusd,Bridged TrueUSD
9023,true-usd,tusd,TrueUSD


Criptomonedas seleccionadas para el análisis.

In [10]:
criptos_order=['Bitcoin','Ethereum','BNB','Cardano','Solana','Polkadot','Bitcoin Cash','Avalanche',
               'Cosmos Hub','Hedera']
criptos = df1[df1['name'].isin(criptos_order)]
criptos

,id,symbol,name
783,avalanche-2,avax,Avalanche
1108,binancecoin,bnb,BNB
1163,bitcoin,btc,Bitcoin
1172,bitcoin-cash,bch,Bitcoin Cash
1636,cardano,ada,Cardano
2083,cosmos,atom,Cosmos Hub
3119,ethereum,eth,Ethereum
4054,hedera-hashgraph,hbar,Hedera
6959,polkadot,dot,Polkadot
8082,solana,sol,Solana


In [11]:
criptos['symbol']

783     avax
1108     bnb
1163     btc
1172     bch
1636     ada
2083    atom
3119     eth
4054    hbar
6959     dot
8082     sol
Name: symbol, dtype: object

### Generando Datos para las diferentes Criptos Seleccionadas:

Obtenga datos de mercado históricos que incluyen precio, capitalización de mercado y volumen de 24 horas dentro de un rango de marca de tiempo (granularidad automática)

In [12]:
def get_coin_market_chart_range_by_id(id:str,markert_chart:str,to:str,From:str):
    ''' Ejem: id = Bitcoin , markert_chart = usd, to = 1660539600, from = 1692079200 '''
    return cg.get_coin_market_chart_range_by_id(id,markert_chart,to,From)

In [13]:
from datetime import datetime
#1660971600 ----> 2022-08-20
#1692424800 ----> 2023-08-19

timestamp = 1692424800

# Convertir el timestamp a una fecha legible
fecha_legible = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

print(fecha_legible)

2023-08-19 06:00:00


In [14]:
def extract_unix_timestamp(item):
    fecha_ilegible= item[0] // 1000  # Dividir por 1000 para obtener 10 dígitos
    fecha_legible = datetime.utcfromtimestamp(fecha_ilegible).strftime('%Y-%m-%d')
    return fecha_legible

In [15]:
def remove_unix_timestamp(item):
    return item[1]

In [16]:
#Está lista se ocupara para que todos los DataFrames tengan las mismas fechas.
fecha_inicio = '2022-08-20'
fecha_fin = '2023-08-19'

rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

# Crear un DataFrame con la columna de fechas
df_Fechas = pd.DataFrame({'fechas': rango_fechas})

In [62]:
def df_market(id:str):    
    #Creación DataFrame
    gcmcrange=get_coin_market_chart_range_by_id(id,'usd','1660971600','1692424800')
    df=pd.DataFrame(gcmcrange)
    #Extracción Fechas:
    df['fechas'] = df['prices'].apply(extract_unix_timestamp)
    #Remove Unix Time
    for i in df:
        df[i+'2'] = df[i].apply(remove_unix_timestamp)
    #Formato DataTime para posterior merge:
    df['fechas'] = pd.to_datetime(df['fechas'])

    columnas_a_eliminar = ['prices', 'market_caps','total_volumes','fechas2']
    df=df.drop(columns= columnas_a_eliminar)

    df_union = pd.merge(df_Fechas, df, on='fechas', how='outer')

    #Agregando Columna con ID 
    df_union.insert(0, 'Cripto', id)
    

    return df_union

#### Categoría: Layer 1 (L1)

In [63]:
criptos

,id,symbol,name
783,avalanche-2,avax,Avalanche
1108,binancecoin,bnb,BNB
1163,bitcoin,btc,Bitcoin
1172,bitcoin-cash,bch,Bitcoin Cash
1636,cardano,ada,Cardano
2083,cosmos,atom,Cosmos Hub
3119,ethereum,eth,Ethereum
4054,hedera-hashgraph,hbar,Hedera
6959,polkadot,dot,Polkadot
8082,solana,sol,Solana


In [64]:
df_Bitcoin_market=df_market('bitcoin')
df_Ethereum_market=df_market('ethereum')
df_BNB_market=df_market('binancecoin')
df_Cardano_market=df_market('cardano')
df_Solana_market=df_market('solana')
df_polkadot_market=df_market('polkadot')
df_bitcoin_cash_market=df_market('bitcoin-cash')
df_Avalanche_market=df_market('avalanche-2')
df_cosmos_market=df_market('cosmos')
df_hedera_hashgraph_market=df_market('hedera-hashgraph')

In [75]:
# Lista de DataFrames a agregar
dataframes = [df_Bitcoin_market, df_Ethereum_market,df_BNB_market,df_Cardano_market,
              df_Solana_market,df_polkadot_market,df_bitcoin_cash_market,df_Avalanche_market
              ,df_cosmos_market,df_hedera_hashgraph_market]

df_concatenated = pd.concat(dataframes, ignore_index=False)

In [77]:
df_concatenated.shape

(3650, 5)

### Extracción de precios de apertura, mayor precio, menor y cierrre.

In [78]:
def Candles_Cripto(id):

    df = pd.DataFrame(client.get_historical_klines(id,'1d','1660971600','1692424800'))
    df = df.iloc[:,:5]
    df.columns = ['fechas','Open','High','Low','Close',]
    df['fechas'] = pd.to_datetime(df.fechas,unit='ms')
    df = df.set_index('fechas')
    df = df.astype(float)
    return df

In [79]:
Symbol=(criptos['symbol'].tolist())

In [80]:
Symbol
for i in range(len(Symbol)):
    Symbol[i] = Symbol[i] + 'usdt'
Symbol = [item.upper() for item in Symbol]

In [81]:
criptos

,id,symbol,name
783,avalanche-2,avax,Avalanche
1108,binancecoin,bnb,BNB
1163,bitcoin,btc,Bitcoin
1172,bitcoin-cash,bch,Bitcoin Cash
1636,cardano,ada,Cardano
2083,cosmos,atom,Cosmos Hub
3119,ethereum,eth,Ethereum
4054,hedera-hashgraph,hbar,Hedera
6959,polkadot,dot,Polkadot
8082,solana,sol,Solana


In [82]:
Symbol

['AVAXUSDT',
 'BNBUSDT',
 'BTCUSDT',
 'BCHUSDT',
 'ADAUSDT',
 'ATOMUSDT',
 'ETHUSDT',
 'HBARUSDT',
 'DOTUSDT',
 'SOLUSDT']

In [83]:
df_Candles_Bitcoin=Candles_Cripto('BTCUSDT')
df_Candles_Ethereum=Candles_Cripto('ETHUSDT')
df_Candles_BNB=Candles_Cripto('BNBUSDT')
df_Candles_Cardano=Candles_Cripto('ADAUSDT')
df_Candles_Solana=Candles_Cripto('SOLUSDT')
df_Candles_polkadot=Candles_Cripto('DOTUSDT')
df_Candles_bitcoin_cash=Candles_Cripto('BCHUSDT')
df_Candles_Avalanche=Candles_Cripto('AVAXUSDT')
df_Candles_cosmos=Candles_Cripto('ATOMUSDT')
df_Candles_hedera_hashgraph=Candles_Cripto('HBARUSDT')

In [85]:
def merge_data(df1,df2):
    df1.set_index('fechas', inplace=True)
    df=df1.merge(df2,left_index=True,right_index=True)
    return df

In [48]:
#merged_df = df_Bitcoin_market.merge(df_Candles_Bitcoin, left_index=True, right_index=True)


In [86]:
df_Bitcoin=merge_data(df_Bitcoin_market,df_Candles_Bitcoin)


In [88]:
df_Ethereum=merge_data(df_Ethereum_market,df_Candles_Ethereum)
df_BNB=merge_data(df_BNB_market,df_Candles_BNB)
df_Cardano=merge_data(df_Cardano_market,df_Candles_Cardano)
df_Solana=merge_data(df_Solana_market,df_Candles_Solana)
df_Polkadot=merge_data(df_polkadot_market,df_Candles_polkadot)
df_Avalanche=merge_data(df_Avalanche_market,df_Candles_Avalanche)
df_Cosmos=merge_data(df_cosmos_market,df_Candles_cosmos)
df_Hedera=merge_data(df_hedera_hashgraph_market,df_Candles_hedera_hashgraph)
df_Bitcoin_Cash=merge_data(df_bitcoin_cash_market,df_Candles_bitcoin_cash)


In [90]:
df_Bitcoin_Cash.shape

(364, 8)

In [91]:
Layer_1=[df_Bitcoin,df_Ethereum,df_BNB,df_Cardano,df_Solana,df_Polkadot,df_Avalanche
            ,df_Cosmos,df_Hedera,df_Bitcoin_Cash]

Layer_1=pd.concat(Layer_1,ignore_index=False)

In [93]:
#Asignar Ranking de acuerdo al MarketCap
latest_data = Layer_1.groupby('Cripto').apply(lambda group: group[group.index == group.index.max()])
latest_data = latest_data.sort_values(by='market_caps2', ascending=False)
latest_data['Ranking'] = range(1, len(latest_data) + 1)
Layer_1['Ranking'] = Layer_1['Cripto'].map(latest_data.set_index('Cripto')['Ranking'])

Agregar Body Ratio

In [96]:
# Calcular el Body Ratio
Layer_1['Body_Ratio'] = abs(Layer_1['Close'] - Layer_1['Open']) / (Layer_1['High'] - Layer_1['Low'])

# Clasificar las velas en tres categorías basadas en el Body Ratio y dirección de cierre
def classify_vela(row):
    if row['Body_Ratio'] < 0.3:
        return 'Indecisión'
    elif row['Body_Ratio'] >= 0.3 and row['Body_Ratio'] <= 0.7:
        return 'Fuerte Volatilidad'
    else:
        if row['Close'] > row['Open']:
            return 'Fuerte Dirección Alcista'
        else:
            return 'Fuerte Dirección Bajista'

Layer_1['Clasificación'] = Layer_1.apply(classify_vela, axis=1)

Agregar Retorno Diario

In [99]:
Layer_1['Daily_Return'] = Layer_1.groupby('Cripto')['Close'].pct_change()

In [101]:
Layer_1['Daily_Return'].fillna(0, inplace=True)

In [103]:
Layer_1.to_csv('DataCriptos.csv',index=True)

Obtención de datos del Top 3 compañias en el sector de Tecnología

Fuente Yahoo.

AAPL	Apple Inc.
MSFT	Microsoft Corporation
NVIDIA Corporation


In [104]:
start_date = "2022-08-20"
end_date = "2023-08-19"
columns=['Dividends','Stock Splits']

In [105]:
AAPL = yf.Ticker("AAPL")
df_AAPL = AAPL.history(period="1d", start=start_date, end=end_date)
df_AAPL = AAPL.history(period="1d", start=start_date, end=end_date)
df_AAPL.index = df_AAPL.index.date
df_AAPL=df_AAPL.drop(columns=columns)
start_date = df_AAPL.index.min()
end_date = df_AAPL.index.max()
date_range = pd.date_range(start=start_date, end=end_date)
df_AAPL = df_AAPL.reindex(date_range)
df_AAPL.insert(0, 'Company', 'Apple Inc.')
df_AAPL=df_AAPL.interpolate()

In [106]:
MSFT = yf.Ticker("MSFT")
df_MSFT = MSFT.history(period="1d", start=start_date, end=end_date)
df_MSFT = MSFT.history(period="1d", start=start_date, end=end_date)
df_MSFT.index = df_MSFT.index.date
df_MSFT=df_MSFT.drop(columns=columns)
start_date = df_MSFT.index.min()
end_date = df_MSFT.index.max()
date_range = pd.date_range(start=start_date, end=end_date)
df_MSFT = df_MSFT.reindex(date_range)
df_MSFT.insert(0, 'Company', 'Microsoft Corporation')
df_MSFT=df_MSFT.interpolate()


In [107]:
DataTec=[df_AAPL,df_MSFT]

DataTec=pd.concat(DataTec,ignore_index=False)

In [109]:
# Calcular el Body Ratio
DataTec['Body_Ratio'] = abs(DataTec['Close'] - DataTec['Open']) / (DataTec['High'] - DataTec['Low'])

DataTec['Clasificación'] = DataTec.apply(classify_vela, axis=1)

Retorno Diario

In [111]:
DataTec['Daily_Return'] = DataTec.groupby('Company')['Close'].pct_change()

In [113]:
DataTec['Daily_Return'].fillna(0, inplace=True)

In [114]:
DataTec.to_csv('DataTec.csv',index=True)